In [111]:
import torch, gzip
from tqdm import trange
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [213]:
image_size = 28

train_images = torch.frombuffer(gzip.open('train-images-idx3-ubyte.gz','r').read(), requires_grad=False, dtype=torch.uint8).float()[16:].view(-1,784)
train_labels = torch.frombuffer(gzip.open('train-labels-idx1-ubyte.gz','r').read(), requires_grad=False, dtype=torch.uint8).int()[8:]
test_images = torch.frombuffer(gzip.open('t10k-images-idx3-ubyte.gz','r').read(), requires_grad=False, dtype=torch.uint8).float()[16:].view(-1,784)
test_labels = torch.frombuffer(gzip.open('t10k-labels-idx1-ubyte.gz','r').read(), requires_grad=False, dtype=torch.uint8).int()[8:]

n = int(0.9 * train_images.shape[0])
X_tr, Y_tr = train_images[:n], train_labels[:n]
X_val, Y_val = train_images[n:], train_labels[n:]
X_test, Y_test = test_images, test_labels

In [216]:
class BroNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.block = nn.Sequential(
            nn.Linear(784, 128),
            nn.Tanh(),
            nn.Linear(128, 128),
            nn.Tanh(),
            nn.Linear(128, 10),
            nn.Tanh()
        )
    def forward(self, x):
        return self.block(x)

model = BroNet()
batch_size = 32
total_steps = 10000
lr = 0.001
optim = torch.optim.SGD(model.parameters(), lr)
def train():
    for i in (t:=trange(total_steps)):
        samp = torch.randint(0, X_tr.shape[0], (batch_size,))
        logits = model(X_tr[samp])
        targets = F.one_hot(Y_tr[samp].long(), num_classes = 10).float()
        loss = F.cross_entropy(logits, targets)
        optim.zero_grad()
        loss.backward()
        optim.step()
        t.set_description(f'loss {loss}')
        break

# def eval(mode):
#         x, y = {
#              'train': (X_tr, Y_tr),
#              'val': (X_val, Y_val),
#              'test': (X_test, Y_test)
#         }[mode]
#         logits = model(x)
#         targets = F.one_hot(y.long())
#         loss = torch.mean((logits - targets)**2)
#         acc = sum(torch.argmax(logits, dim=1) == y) / logits.shape[0]

#         print(f'loss {loss:8.2f} accuracy {acc:8.2f}')

train()
eval('val')


loss 2.258934736251831:   0%|          | 0/10000 [00:00<?, ?it/s]

loss     0.15 accuracy     0.09


  0%|          | 0/10000 [00:00<?, ?it/s]

torch.Size([32, 10])


TypeError: unsupported format string passed to Tensor.__format__